In [1]:
#import nltk
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
#from nltk.tokenize import sent_tokenize
#from nltk.stem import PorterStemmerfrom sklearn.feature_extraction.text 

#from sklearn.feature_extraction.text 
#import CountVectorizer
from sklearn.linear_model  import LogisticRegressionCV
#from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#peace=0: low peace;  peace=1:high peace
ca = pd.read_csv("CA_word.csv")
us = pd.read_csv("US_word.csv")
sg = pd.read_csv("SG_word.csv")
ph = pd.read_csv("PH_word.csv")

In [3]:
ca.head()

,Unnamed: 0,word,frequency,relative_freq
0,0,the,2913900,0.083373
1,1,to,1453643,0.041592
2,2,and,1388210,0.039720
3,3,of,1344326,0.038464
4,4,a,1199482,0.034320


In [4]:
len(ca)

1000

In [5]:
ca['peace']=1

In [6]:
ca.head()

,Unnamed: 0,word,frequency,relative_freq,peace
0,0,the,2913900,0.083373,1
1,1,to,1453643,0.041592,1
2,2,and,1388210,0.039720,1
3,3,of,1344326,0.038464,1
4,4,a,1199482,0.034320,1


In [7]:
us['peace']=0
sg['peace']=1
ph['peace']=0

## 1. CA vs. US

In [8]:
#CA vs US
frames=[ca,us]
ca_us = pd.concat(frames)

In [9]:
len(ca_us)

2000

### Data splittiing

In [10]:
X = ca_us[['relative_freq']]
y = ca_us[['peace']]

In [11]:
X_dev,X_test,y_dev,y_test=train_test_split(X,y,
                                          test_size=0.2,random_state=42)

In [12]:
y_dev.value_counts()

peace
1        801
0        799
dtype: int64

In [13]:
y_test.value_counts()

peace
0        201
1        199
dtype: int64

### Resample data

In [14]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_dev, y_dev = smote.fit_resample(X_dev, y_dev)
X_test, y_test = smote.fit_resample(X_test, y_test)

In [15]:
y_dev.value_counts()

peace
0        801
1        801
dtype: int64

In [16]:
y_test.value_counts()

peace
0        201
1        201
dtype: int64

In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from  sklearn.linear_model import LogisticRegression

### (1)  Logistic Regression

In [18]:
lr = LogisticRegression()
clf_lr_def = lr.fit(X_dev, y_dev.values.ravel())
scores_lr = cross_val_score(lr, X_dev, y_dev.values.ravel(), cv = 5)

In [19]:

print("The score for Logistic Regression on devlopment data is %f." %(scores_lr.mean()))
print("The score for Logistic Regression on test data is %f." %(clf_lr_def.score(X_test, y_test)))

The score for Logistic Regression on devlopment data is 0.495002.
The score for Logistic Regression on test data is 0.507463.


In [20]:
y_hat_test = clf_lr_def.predict(X_test)

In [21]:
from sklearn import metrics
cm  = metrics.confusion_matrix(y_test,y_hat_test)
print(cm)

[[179  22]
 [176  25]]


In [22]:
lr_cv = cross_val_score(lr, X_dev, y_dev.values.ravel(), cv=5, scoring = 'accuracy')
print('Logistic Regression mean: %f ' % lr_cv.mean())

Logistic Regression mean: 0.495002 


In [23]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_hat_test, digits=6))

              precision    recall  f1-score   support

           0   0.504225  0.890547  0.643885       201
           1   0.531915  0.124378  0.201613       201

    accuracy                       0.507463       402
   macro avg   0.518070  0.507463  0.422749       402
weighted avg   0.518070  0.507463  0.422749       402



### (2) Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
rf = RandomForestClassifier()
clf_rf = rf.fit(X_dev, y_dev.values.ravel())
scores_rf = cross_val_score(rf, X_dev, y_dev.values.ravel(), cv = 5)

print("The score for Random Forest on devlopment data is %f." %(scores_rf.mean()))
print("The score for Random Forest on test data is %f." %(clf_rf.score(X_test, y_test)))

The score for Random Forest on devlopment data is 0.492529.
The score for Random Forest on test data is 0.500000.


In [26]:
y_test_rf_hat = clf_rf.predict(X_test)
print(classification_report(y_test,y_test_rf_hat, digits=4))

              precision    recall  f1-score   support

           0     0.5000    0.5124    0.5061       201
           1     0.5000    0.4876    0.4937       201

    accuracy                         0.5000       402
   macro avg     0.5000    0.5000    0.4999       402
weighted avg     0.5000    0.5000    0.4999       402



###  (3) SVM

In [27]:
from sklearn.svm import LinearSVC, SVC

In [28]:
svc = SVC()
svc.fit(X_dev, y_dev.values.ravel())
scores_svm = cross_val_score(svc, X_dev, y_dev.values.ravel(), cv=5, scoring='accuracy')


In [29]:
print('Accuracy on development data: %f' % scores_svm.mean())
print('Accuracy on test data: %f' % svc.score(X_test, y_test))

Accuracy on development data: 0.488754
Accuracy on test data: 0.495025


In [30]:
y_hat_svc = svc.predict(X_test)
print(classification_report(y_test, y_hat_svc, digits=4))

              precision    recall  f1-score   support

           0     0.4375    0.0348    0.0645       201
           1     0.4974    0.9552    0.6542       201

    accuracy                         0.4950       402
   macro avg     0.4675    0.4950    0.3593       402
weighted avg     0.4675    0.4950    0.3593       402



## 2. SG vs. PH

In [31]:
frames=[sg,ph]
sg_ph = pd.concat(frames)

In [32]:
X = sg_ph[['relative_freq']]
y = sg_ph[['peace']]

In [33]:
X_dev,X_test,y_dev,y_test=train_test_split(X,y,
                                          test_size=0.2,random_state=42)

In [34]:
y_dev.value_counts()

peace
1        801
0        799
dtype: int64

In [35]:
y.value_counts()

peace
0        1000
1        1000
dtype: int64

In [36]:
X_dev, y_dev = smote.fit_resample(X_dev, y_dev)
X_test, y_test = smote.fit_resample(X_test, y_test)

### (1) Logistic Regression

In [37]:
scores_lr = cross_val_score(lr, X_dev, y_dev.values.ravel(), cv = 5)
clf_lr_def = lr.fit(X_dev, y_dev.values.ravel())
print("The score for Logistic Regression on devlopment data is %f." %(scores_lr.mean()))
print("The score for Logistic Regression on test data is %f." %(clf_lr_def.score(X_test, y_test)))

The score for Logistic Regression on devlopment data is 0.493752.
The score for Logistic Regression on test data is 0.507463.


In [38]:
y_hat_test = clf_lr_def.predict(X_test)
cm  = metrics.confusion_matrix(y_test,y_hat_test)
print(cm)

[[182  19]
 [179  22]]


In [39]:
lr_cv = cross_val_score(lr, X_dev, y_dev.values.ravel(), cv=5, scoring = 'accuracy')
print('Logistic Regression mean: %f ' % lr_cv.mean())

Logistic Regression mean: 0.493752 


In [40]:
print(classification_report(y_test,y_hat_test, digits=6))

              precision    recall  f1-score   support

           0   0.504155  0.905473  0.647687       201
           1   0.536585  0.109453  0.181818       201

    accuracy                       0.507463       402
   macro avg   0.520370  0.507463  0.414753       402
weighted avg   0.520370  0.507463  0.414753       402



### (2) Random Forest

In [41]:
clf_rf = rf.fit(X_dev, y_dev.values.ravel())
scores_rf = cross_val_score(rf, X_dev, y_dev.values.ravel(), cv = 5)

print("The score for Random Forest on devlopment data is %f." %(scores_rf.mean()))
print("The score for Random Forest on test data is %f." %(clf_rf.score(X_test, y_test)))

The score for Random Forest on devlopment data is 0.512457.
The score for Random Forest on test data is 0.467662.


In [42]:
y_test_rf_hat = clf_rf.predict(X_test)
print(classification_report(y_test,y_test_rf_hat, digits=4))

              precision    recall  f1-score   support

           0     0.4683    0.4776    0.4729       201
           1     0.4670    0.4577    0.4623       201

    accuracy                         0.4677       402
   macro avg     0.4676    0.4677    0.4676       402
weighted avg     0.4676    0.4677    0.4676       402



### (3) SVM

In [43]:
svc.fit(X_dev, y_dev.values.ravel())
scores_svm = cross_val_score(svc, X_dev, y_dev.values.ravel(), cv=5, scoring='accuracy')


In [44]:
print('Accuracy on development data: %f' % scores_svm.mean())
print('Accuracy on test data: %f' % svc.score(X_test, y_test))

Accuracy on development data: 0.489381
Accuracy on test data: 0.572139


In [45]:
y_hat_svc = svc.predict(X_test)
print(classification_report(y_test, y_hat_svc, digits=4))

              precision    recall  f1-score   support

           0     0.5775    0.5373    0.5567       201
           1     0.5674    0.6070    0.5865       201

    accuracy                         0.5721       402
   macro avg     0.5725    0.5721    0.5716       402
weighted avg     0.5725    0.5721    0.5716       402



## 3. CA, SG vs. US,PH

In [46]:
frames=[ca,us,sg,ph]
df = pd.concat(frames)

In [47]:
X = df[['relative_freq']]
y = df[['peace']]

In [48]:
y.value_counts()

peace
0        2000
1        2000
dtype: int64

In [49]:
X_dev,X_test,y_dev,y_test=train_test_split(X,y,
                                          test_size=0.2,random_state=42)

In [50]:
y_dev.value_counts()

peace
0        1603
1        1597
dtype: int64

In [51]:
X_dev, y_dev = smote.fit_resample(X_dev, y_dev)
X_test, y_test = smote.fit_resample(X_test, y_test)

In [52]:
y_dev.value_counts()

peace
0        1603
1        1603
dtype: int64

### (1) Logistic Regression

In [53]:
scores_lr = cross_val_score(lr, X_dev, y_dev.values.ravel(), cv = 5)
clf_lr_def = lr.fit(X_dev, y_dev.values.ravel())
print("The score for Logistic Regression on devlopment data is %f." %(scores_lr.mean()))
print("The score for Logistic Regression on test data is %f." %(clf_lr_def.score(X_test, y_test)))

The score for Logistic Regression on devlopment data is 0.496883.
The score for Logistic Regression on test data is 0.501241.


In [54]:
y_hat_test = clf_lr_def.predict(X_test)
cm  = metrics.confusion_matrix(y_test,y_hat_test)
print(cm)

[[ 54 349]
 [ 53 350]]


In [55]:
lr_cv = cross_val_score(lr, X_dev, y_dev.values.ravel(), cv=5, scoring = 'accuracy')
print('Logistic Regression mean: %f ' % lr_cv.mean())

Logistic Regression mean: 0.496883 


In [56]:
print(classification_report(y_test,y_hat_test, digits=6))

              precision    recall  f1-score   support

           0   0.504673  0.133995  0.211765       403
           1   0.500715  0.868486  0.635209       403

    accuracy                       0.501241       806
   macro avg   0.502694  0.501241  0.423487       806
weighted avg   0.502694  0.501241  0.423487       806



### (2) Random Forest

In [57]:
clf_rf = rf.fit(X_dev, y_dev.values.ravel())
scores_rf = cross_val_score(rf, X_dev, y_dev.values.ravel(), cv = 5)

print("The score for Random Forest on devlopment data is %f." %(scores_rf.mean()))
print("The score for Random Forest on test data is %f." %(clf_rf.score(X_test, y_test)))

The score for Random Forest on devlopment data is 0.521201.
The score for Random Forest on test data is 0.507444.


In [58]:
y_test_rf_hat = clf_rf.predict(X_test)
print(classification_report(y_test,y_test_rf_hat, digits=4))

              precision    recall  f1-score   support

           0     0.5070    0.5360    0.5211       403
           1     0.5079    0.4789    0.4930       403

    accuracy                         0.5074       806
   macro avg     0.5075    0.5074    0.5070       806
weighted avg     0.5075    0.5074    0.5070       806



### (3) SVM

In [59]:
svc.fit(X_dev, y_dev.values.ravel())
scores_svm = cross_val_score(svc, X_dev, y_dev.values.ravel(), cv=5, scoring='accuracy')


In [60]:
print('Accuracy on development data: %f' % scores_svm.mean())
print('Accuracy on test data: %f' % svc.score(X_test, y_test))

Accuracy on development data: 0.494074
Accuracy on test data: 0.492556


In [61]:
y_hat_svc = svc.predict(X_test)
print(classification_report(y_test, y_hat_svc, digits=4))

              precision    recall  f1-score   support

           0     0.4959    0.9057    0.6409       403
           1     0.4571    0.0794    0.1353       403

    accuracy                         0.4926       806
   macro avg     0.4765    0.4926    0.3881       806
weighted avg     0.4765    0.4926    0.3881       806

